## Name: "Mingrui Liu"

# Computer Assignment: DTMF Decoder

Build a DTMF decoder.  Your code should accept a tonal sequence and output a string of key presses.

Rules:

1. Use a sampling rate of 8000 samples per second.
1. Use a set of bandpass filters, not a FFT.  
1. For full credit, use FIR filters of length 31 or less or second order (two pole) IIR filters.  You get partial credit for working implementations that use larger filters.
1. You may assume the silence and tonal periods are both multiples of 400 samples.  (This is a simplifying assumption.) 
1. You may not assume the tones or the silence periods are the same length.

Due Friday, Oct 20.

In [81]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rnd
from IPython.display import Audio
%matplotlib inline

## Reference DTMF encoder

Here is a reference DTMF encoder (and the solution to the first part of the assignment).

In [82]:
Fs = 8000

#define the keypad and its frequencies
validkeys = '*#0123456789ABCD'
rowfreqs = [697, 770, 852, 941]
colfreqs = [1209, 1336, 1477, 1633]
buttons = {'1':(0,0), '2':(0,1), '3':(0,2), 'A':(0,3),
           '4':(1,0), '5':(1,1), '6':(1,2), 'B':(1,3),
           '7':(2,0), '8':(2,1), '9':(2,2), 'C':(2,3),
           '*':(3,0), '0':(3,1), '#':(3,2), 'D':(3,3)}

def dtmf_encoder(phonenumber, dur = 0.5, silencedur=0.1, Fs=8000):
    """return the DTMF tones for a phone number"""
    t = np.linspace(0,dur,int(dur*Fs),endpoint=False)
    silence = np.zeros(int(silencedur*Fs))
    
    sounds = []
    for key in phonenumber:
        if key.upper() in validkeys:
            r,c = buttons[key]
            fr, fc = rowfreqs[r], colfreqs[c]
            #print key, fr, fc
            sounds.append(np.sin(2*np.pi*fr*t)+np.sin(2*np.pi*fc*t))
            sounds.append(silence)
    return np.concatenate(sounds[:-1]) #drop last silence period   

In [83]:
myphone = '302-831-8008'
Audio(dtmf_encoder(myphone,silencedur=0.5),rate=Fs)

In [84]:
help = '911'
Audio(dtmf_encoder(help, dur=1,silencedur=0),rate=Fs)

In [85]:
test = '123A456B789C*0#D'
Audio(dtmf_encoder(test), rate=Fs)

In [86]:
test2 = '147*2580369#ABCD'
Audio(dtmf_encoder(test2), rate=Fs)

## DTMF Decoder

Put your solution here.

In [87]:
def DTMF_decode(audio, dur = 0.5, silencedur = 0.1, Fs = 8000):
    def Tones(audio, Fs):
        audio_wav = audio.astype('double') / (2 ** 15 - 1)
        x = np.arange(len(audio_wav)) / Fs
        tones = []
        P, freqz = plt.specgram(audio, Fs)
        for blk in range(len(P[0])):
            block = np.array(list(zip(P))[blk])[:60]
            block, freq = resample(block, 10*len(block), freqz[:60] )
            peak = find_peak(block[:len(block)/1.8], np.arange(3, 20))
            peak = np.array(peak)
            if len(peak) and np.max(block[peak]) > 100:
                tone = Tone(block * 512 / Fs, ((block + 1) * 512 - 1)/Fs, {freq[freqs]: block[freqs] for freqs in peak},)
                tones.append(tone)
                
        return tones
    def get_number(freqs):
        colfreqs = 0
        colfreqs_v = 0
        for k in colfreqs:
            if freqs[k] > colfreqs_v:
                colfreqs_v = freqs[k]
                colfreqs = k
        rowfreqs = 0
        roefreqs_v = 0
        for k in rowfreqs:
            if freqs[k] > rowfreqs_v:
                rowfreqs_v = freqs[k]
                rowfreqs = k
        if colfreqs == 1209:
            if rowfreqs == 697:
                return "1"
            if rowfreqs == 770:
                return "4"
            if rowfreqs == 852:
                return "7"
            if rowfreqs == 941:
                return "*"
        elif colfreqs == 1336:
            if rowfreqs == 697:
                return "2"
            if rowfreqs == 770:
                return "5"
            if rowfreqs == 852:
                return "8"
            if rowfreqs == 941:
                return "0"
        elif colfreqs == 1477: 
            if rowfreqs == 697:
                return "3"
            if rowfreqs == 770:
                return "6"
            if rowfreqs == 852:
                return "9"
            if rowfreqs == 941:
                return "#"
        elif colfreqs == 1633: 
            if rowfreqs == 697:
                return "A"
            if rowfreqs == 770:
                return "B"
            if rowfreqs == 852:
                return "C"
            if rowfreqs == 941:
                return "D"

    pass

## Automatially test the decoder

`test_dtmfdecoder` iterates over increasing noise until the `dtmfdecoder` routine fails.  It runs each level 5 times.  We reserve the right to change the default values for `dur` and `silencedur`; do not assume the current defaults will always be used.

In [88]:
def test_dtmfdecoder(dtmfdecoder, dur=0.5, silencedur=0.1, Fs=8000 ):
    works = False
    sigma = 0.0
    while sigma < 3.1:
        number = ''.join([c for c in rnd.permutation(list(validkeys))])
        #print number
        tones = dtmf_encoder(number, dur=dur, silencedur=silencedur)
        
        for i in range(5):
            noisy_tones = tones + sigma*rnd.randn(len(tones))
            decoded = dtmfdecoder(noisy_tones)
            if decoded == number:
                works = True
                #print sigma, i
            else:
                return works, sigma
        sigma += 0.1
    return works, sigma
        

In [89]:
test_dtmfdecoder(DTMF_decode) #my solution

(False, 0.0)